In [1]:
# Configuration for auto-completion
%config Completer.use_jedi = False
%config Completer.greedy = True

# Enable test mode for CI pipeline
import sys
if len(sys.argv) == 2 and sys.argv[1] == "test":
    test_mode = True
else:
    test_mode = False


In [2]:
import time
from qciconnect import Client

# Load environment variables for logging in to  QCI Connect
# Insert your credentials below
user=%env CI_KEYCLOAK_USER
pwd=%env CI_KEYCLOAK_PW

In [3]:
client = Client(username=user, password=pwd, server="https://qc-plat-pro.dlr.de/")

In [4]:
client.methods.show()

transpilation_qiskit_compiler


## Compile Job Submission

In [5]:
circuit = """OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];"""

In [6]:
result = client.methods.transpilation_qiskit_compiler({"circuit": circuit})

Job submitted with ID: 80b18ba3-eb06-4a45-b52e-742f404ff4a2
Job was successful.


In [7]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];



In [8]:
result = client.methods.transpilation_qiskit_compiler({"circuit": circuit}, {
      "qubit_connectivity": [[0,1]],
      "gate_set": ["h", "x", "cz"],
      "optimization_level": 2
    })

Job submitted with ID: f7019eff-1a2b-4c88-b863-4a5328a68a4b
Job was successful.


In [9]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
h $0;
h $1;
cz $0, $1;
h $0;



## Non-blocking compile job submission

In [10]:
promise = client.methods.transpilation_qiskit_compiler({"circuit": circuit}, wait_for_results=False)


Job submitted with ID: 5107b6e8-e9f3-4215-a6c1-4718aa4f6fa3
Job was successful.


In [11]:
if test_mode:
    print("Waiting in test mode for job to complete...")
    time.sleep(5)
    
result = promise.result

In [12]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];



## QPU Job Submission

In [13]:
client.qpus.show()

Alias                    Qubits  Status
---------------------  --------  -----------
qsea_i_|_digital_twin        10  offline
qsea_i                       10  offline
qsea_ii                      20  coming_soon
xq1i                          3  offline
xq2i                          8  coming_soon
redac                         8  coming_soon
dinaqc                      100  coming_soon
legato                       20  coming_soon
toccata                      50  coming_soon
xaphiro                      50  coming_soon
sunqc                         8  coming_soon
carina                        8  offline
dedalo                       64  coming_soon
comiqc                       50  coming_soon
statevector_simulator        24  running
saxonq_4qb_rt                 3  running


In [14]:
result = client.qpus.statevector_simulator.submit(circuit, "sampling", shots=10, name="API Demonstration", comment="Submit algorithm X with parameters YZ")

Job submitted with ID: d5765812-de40-49c1-abf3-16f21e35d5d9
Job was successful.


In [15]:
print(result.data)

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]


## Non-blocking QPU job submission

In [16]:
promise = client.qpus.statevector_simulator.submit(circuit, "sampling", shots=10, wait_for_results=False, name="API Demonstration", comment="Submit algorithm X with parameters YZ")


Job submitted with ID: 43f446c6-520b-4203-8453-f4778ed8e2a4
Job was successful.


In [17]:
if test_mode:
    print("Waiting in test mode for job to complete...")
    time.sleep(5)
    
result = promise.result
print(result.data)

[[1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]


In [18]:
if not test_mode:
    promise = client.qpus.saxonq_4qb_rt.submit(circuit, "sampling", name="Default circuit on SaxonQ machine", shots=600000, wait_for_results=False)
    

Job submitted with ID: 1df2e240-b850-41e7-bca1-c467b62c7fed
Job is not finished yet.


In [19]:
if not test_mode:
    result = promise.result

Job is not finished yet.


In [ ]:
print(result.data)